In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

24393


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
24388,2021-06-13,47,沖縄県,104,19508,0,159
24389,2021-06-14,47,沖縄県,54,19562,4,163
24390,2021-06-15,47,沖縄県,107,19669,0,163
24391,2021-06-16,47,沖縄県,115,19784,0,163
24392,2021-06-17,47,沖縄県,97,19881,4,167


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,11418,8959,145
43,香川県,19203,2078,29
44,高知県,20241,1699,23
45,鳥取県,16089,466,2
46,鹿児島県,23874,3621,33


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
24386,2021-06-11,47,沖縄県,145,19247,0,156,1192.0
24387,2021-06-12,47,沖縄県,157,19404,3,159,1088.0
24388,2021-06-13,47,沖縄県,104,19508,0,159,1009.0
24389,2021-06-14,47,沖縄県,54,19562,4,163,959.0
24390,2021-06-15,47,沖縄県,107,19669,0,163,907.0
24391,2021-06-16,47,沖縄県,115,19784,0,163,848.0
24392,2021-06-17,47,沖縄県,97,19881,4,167,779.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
24383,2021-06-08,47,沖縄県,159,18762,0,154,1495.0,0.528846,0.002952,-0.286996
24384,2021-06-09,47,沖縄県,174,18936,2,156,1372.0,0.094340,-0.030976,-0.414141
24385,2021-06-10,47,沖縄県,166,19102,0,156,1294.0,-0.045977,-0.012052,-0.319672
24386,2021-06-11,47,沖縄県,145,19247,0,156,1192.0,-0.126506,-0.031880,-0.412955
24387,2021-06-12,47,沖縄県,157,19404,3,159,1088.0,0.082759,-0.028155,-0.398467
24388,2021-06-13,47,沖縄県,104,19508,0,159,1009.0,-0.337580,-0.033687,-0.431694
24389,2021-06-14,47,沖縄県,54,19562,4,163,959.0,-0.480769,-0.040698,-0.480769
24390,2021-06-15,47,沖縄県,107,19669,0,163,907.0,0.981481,0.023964,-0.327044
24391,2021-06-16,47,沖縄県,115,19784,0,163,848.0,0.074766,0.021168,-0.339080
24392,2021-06-17,47,沖縄県,97,19881,4,167,779.0,-0.156522,0.005376,-0.415663


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
518,2021-06-17,北海道,73,4,674.0,-0.170455,-0.109439,-0.598901
1037,2021-06-17,青森県,0,0,10.0,-1.000000,NaN,-1.000000
1556,2021-06-17,岩手県,4,0,72.0,-0.600000,-0.103685,-0.714286
2075,2021-06-17,宮城県,8,0,54.0,1.000000,0.117723,-0.111111
2594,2021-06-17,秋田県,0,0,3.0,NaN,NaN,NaN


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,12456,5163,109,2021-06-17,4,0,64.0,-0.428571,-0.046145,-0.777778
1,京都府,13494,16415,234,2021-06-17,17,1,154.0,-0.260870,-0.002919,-0.585366
2,佐賀県,21279,2543,23,2021-06-17,1,0,8.0,-0.666667,NaN,0.000000
3,兵庫県,14532,40650,1270,2021-06-17,39,2,244.0,0.147059,0.048672,-0.220000
4,北海道,519,40794,1331,2021-06-17,73,4,674.0,-0.170455,-0.109439,-0.598901


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,12456,5163,109,2021-06-17,4,0,64.0,-0.428571,-0.046145,-0.777778,1813859
1,京都府,13494,16415,234,2021-06-17,17,1,154.0,-0.260870,-0.002919,-0.585366,2545899
2,佐賀県,21279,2543,23,2021-06-17,1,0,8.0,-0.666667,NaN,0.000000,823810
3,兵庫県,14532,40650,1270,2021-06-17,39,2,244.0,0.147059,0.048672,-0.220000,5549568
4,北海道,519,40794,1331,2021-06-17,73,4,674.0,-0.170455,-0.109439,-0.598901,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,519,40794,1331,2021-06-17,73,4,674.0,-0.170455,-0.109439,-0.598901,5267762,77.440856,774.408563,12.794807
41,青森県,1038,2452,31,2021-06-17,0,0,10.0,-1.000000,NaN,-1.000000,1275783,19.219569,192.195695,0.783832
19,岩手県,1557,1620,46,2021-06-17,4,0,72.0,-0.600000,-0.103685,-0.714286,1235517,13.111920,131.119199,5.827520
11,宮城県,2076,9057,85,2021-06-17,8,0,54.0,1.000000,0.117723,-0.111111,2292385,39.509070,395.090702,2.355625
36,秋田県,2595,764,20,2021-06-17,0,0,3.0,NaN,NaN,NaN,985416,7.753071,77.530708,0.304440
15,山形県,3114,2017,46,2021-06-17,0,0,14.0,-1.000000,0.500000,-1.000000,1082296,18.636307,186.363065,1.293546
35,福島県,3633,4741,158,2021-06-17,9,0,52.0,-0.181818,0.565601,-0.250000,1881981,25.191540,251.915402,2.763046
38,茨城県,4152,10200,160,2021-06-17,15,0,188.0,-0.347826,0.488746,-0.400000,2921436,34.914337,349.143366,6.435191
27,栃木県,4671,6739,80,2021-06-17,11,0,136.0,-0.266667,0.006753,-0.656250,1965516,34.286162,342.861620,6.919303
37,群馬県,5190,7994,149,2021-06-17,2,0,36.0,-0.500000,-0.017007,-0.714286,1969439,40.590239,405.902391,1.827932


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      16544.702128
std       30303.448596
min         466.000000
25%        2497.500000
50%        4949.000000
75%       10758.500000
max      167868.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,6747,167868,2190,2021-06-17,452,7,2705.0,-0.097804,0.057743,0.029613,13834925,121.336400,1213.364005,19.551967
1,大阪府,14013,102144,2567,2021-06-17,95,4,726.0,-0.120370,-0.001053,-0.358108,8849635,115.421709,1154.217095,8.203728
2,神奈川県,7266,64889,933,2021-06-17,185,2,1333.0,-0.119048,0.018946,-0.021164,9209442,70.459209,704.592091,14.474275
3,愛知県,11937,50234,917,2021-06-17,99,7,727.0,-0.100000,0.018569,-0.421053,7575530,66.310872,663.108720,9.596688
4,埼玉県,5709,45314,825,2021-06-17,79,2,512.0,-0.091954,0.002436,-0.281818,7390054,61.317549,613.175492,6.928231
5,北海道,519,40794,1331,2021-06-17,73,4,674.0,-0.170455,-0.109439,-0.598901,5267762,77.440856,774.408563,12.794807
6,兵庫県,14532,40650,1270,2021-06-17,39,2,244.0,0.147059,0.048672,-0.220000,5549568,73.248945,732.489448,4.396739
7,千葉県,6228,38801,699,2021-06-17,124,3,710.0,0.097345,0.050790,0.215686,6319772,61.396202,613.962023,11.234583
8,福岡県,20760,35112,507,2021-06-17,41,7,280.0,0.025000,0.040820,-0.023810,5129841,68.446566,684.465659,5.458259
9,沖縄県,24393,19881,167,2021-06-17,97,4,779.0,-0.156522,0.005376,-0.415663,1481547,134.190815,1341.908154,52.580175


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      39.894096
std       28.903217
min        7.753071
25%       22.217403
50%       30.185133
75%       51.998390
max      134.190815
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,24393,19881,167,2021-06-17,97,4,779.0,-0.156522,0.005376,-0.415663,1481547,134.190815,1341.908154,52.580175
26,東京都,6747,167868,2190,2021-06-17,452,7,2705.0,-0.097804,0.057743,0.029613,13834925,121.336400,1213.364005,19.551967
32,神奈川県,7266,64889,933,2021-06-17,185,2,1333.0,-0.119048,0.018946,-0.021164,9209442,70.459209,704.592091,14.474275
4,北海道,519,40794,1331,2021-06-17,73,4,674.0,-0.170455,-0.109439,-0.598901,5267762,77.440856,774.408563,12.794807
16,山梨県,9861,1881,20,2021-06-17,13,0,104.0,1.166667,0.085431,-0.593750,826579,22.756446,227.564455,12.581979
5,千葉県,6228,38801,699,2021-06-17,124,3,710.0,0.097345,0.050790,0.215686,6319772,61.396202,613.962023,11.234583
24,愛知県,11937,50234,917,2021-06-17,99,7,727.0,-0.100000,0.018569,-0.421053,7575530,66.310872,663.108720,9.596688
44,高知県,20241,1699,23,2021-06-17,10,0,62.0,0.666667,0.180427,-0.090909,709230,23.955557,239.555574,8.741875
9,大阪府,14013,102144,2567,2021-06-17,95,4,726.0,-0.120370,-0.001053,-0.358108,8849635,115.421709,1154.217095,8.203728
7,埼玉県,5709,45314,825,2021-06-17,79,2,512.0,-0.091954,0.002436,-0.281818,7390054,61.317549,613.175492,6.928231


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    35.000000
mean      0.165211
std       0.244110
min      -0.109439
25%       0.003906
50%       0.050790
75%       0.254402
max       0.834921
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
10,奈良県,15051,7994,122,2021-06-17,13,0,77.0,0.444444,0.834921,1.600000,1353837,59.046990,590.469901,5.687538
31,石川県,8823,3894,114,2021-06-17,7,2,33.0,0.750000,0.665646,0.000000,1139612,34.169524,341.695244,2.895722
13,富山県,8304,1985,37,2021-06-17,11,0,37.0,1.200000,0.628571,1.200000,1055999,18.797366,187.973663,3.503791
35,福島県,3633,4741,158,2021-06-17,9,0,52.0,-0.181818,0.565601,-0.250000,1881981,25.191540,251.915402,2.763046
15,山形県,3114,2017,46,2021-06-17,0,0,14.0,-1.000000,0.500000,-1.000000,1082296,18.636307,186.363065,1.293546
38,茨城県,4152,10200,160,2021-06-17,15,0,188.0,-0.347826,0.488746,-0.400000,2921436,34.914337,349.143366,6.435191
14,山口県,18165,3090,71,2021-06-17,0,0,45.0,-1.000000,0.466162,-1.000000,1369882,22.556687,225.566874,3.284954
18,岡山県,17127,7568,123,2021-06-17,4,0,47.0,-0.500000,0.389827,-0.666667,1903627,39.755687,397.556874,2.468971
39,長崎県,21798,3087,69,2021-06-17,10,0,57.0,0.428571,0.285976,-0.230769,1350769,22.853649,228.536486,4.219818
25,新潟県,7785,3382,47,2021-06-17,6,0,51.0,0.200000,0.222827,-0.625000,2236042,15.124940,151.249395,2.280816


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,519,40794,1331,2021-06-17,73,4,674.0,-0.170455,-0.109439,-0.598901,5267762,77.440856,774.408563,12.794807
19,岩手県,1557,1620,46,2021-06-17,4,0,72.0,-0.600000,-0.103685,-0.714286,1235517,13.111920,131.119199,5.827520
0,三重県,12456,5163,109,2021-06-17,4,0,64.0,-0.428571,-0.046145,-0.777778,1813859,28.464175,284.641750,3.528389
46,鹿児島県,23874,3621,33,2021-06-17,3,0,75.0,-0.769231,-0.031210,-0.727273,1630146,22.212734,222.127343,4.600815
37,群馬県,5190,7994,149,2021-06-17,2,0,36.0,-0.500000,-0.017007,-0.714286,1969439,40.590239,405.902391,1.827932
17,岐阜県,10899,9136,183,2021-06-17,13,1,114.0,0.000000,-0.007195,-0.458333,2032490,44.949791,449.497907,5.608884
1,京都府,13494,16415,234,2021-06-17,17,1,154.0,-0.260870,-0.002919,-0.585366,2545899,64.476242,644.762420,6.048944
9,大阪府,14013,102144,2567,2021-06-17,95,4,726.0,-0.120370,-0.001053,-0.358108,8849635,115.421709,1154.217095,8.203728
7,埼玉県,5709,45314,825,2021-06-17,79,2,512.0,-0.091954,0.002436,-0.281818,7390054,61.317549,613.175492,6.928231
28,沖縄県,24393,19881,167,2021-06-17,97,4,779.0,-0.156522,0.005376,-0.415663,1481547,134.190815,1341.908154,52.580175


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   466.    21391.25  42316.5   63241.75  84167.   105092.25 126017.5
 146942.75 167868.  ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([38.,  2.,  5.,  0.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 9., 23.,  4.,  8.,  0.,  1.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="和歌山県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
15560,2021-06-08,30,和歌山県,1,2639,0,45,18.0,0.000000,NaN,-0.666667
15561,2021-06-09,30,和歌山県,2,2641,2,47,12.0,1.000000,NaN,-0.750000
15562,2021-06-10,30,和歌山県,6,2647,0,47,14.0,2.000000,NaN,0.500000
15563,2021-06-11,30,和歌山県,1,2648,0,47,15.0,-0.833333,NaN,inf
15564,2021-06-12,30,和歌山県,3,2651,0,47,16.0,2.000000,0.523810,0.500000
15565,2021-06-13,30,和歌山県,1,2652,0,47,15.0,-0.666667,0.428571,-0.500000
15566,2021-06-14,30,和歌山県,0,2652,1,48,14.0,-1.000000,0.357143,-1.000000
15567,2021-06-15,30,和歌山県,2,2654,0,48,15.0,inf,NaN,1.000000
15568,2021-06-16,30,和歌山県,0,2654,0,48,13.0,-1.000000,NaN,-1.000000
15569,2021-06-17,30,和歌山県,1,2655,0,48,8.0,inf,NaN,-0.833333
